In [3]:
import os
import sys

In [1]:
import json

In [4]:
_PKG_PATH: str = "/Users/adebayobraimah/Desktop/projects/CSE505"
sys.path.append(_PKG_PATH)

In [42]:
from src.clapi.clapi import _write_list_to_file, _translate_range

In [2]:
json_file = (
    "/Users/adebayobraimah/Desktop/projects/CSE505/src/resources/cse_courses.json"
)

In [43]:
def generate_clingo_predicates_rules(json_file):
    with open(json_file, "r") as file:
        data = json.load(file)

    predicates = []
    rules = []

    output_file = json_file.replace(".json", ".lp")

    for course_id, course_info in data.items():
        # Generate course predicates
        offered_spring1 = int(course_info["spring1"])
        offered_fall1 = int(course_info["fall1"])
        offered_spring2 = int(course_info["spring2"])
        offered_fall2 = int(course_info["fall2"])

        credits = (
            int(course_info.get("Credits"))
            if isinstance(course_info.get("Credits"), (float, int))
            else _translate_range(course_info.get("Credits"))
        )

        career = course_info.get("Career")

        predicates.append(
            f'course({course_id.lower()}, {credits}, "{career}", {offered_spring1}, {offered_fall1}, {offered_spring2}, {offered_fall2}).'
        )

        # Generate antirequisite rules
        if course_info["Antirequisites"] != "NONE":
            rules.append(f"% Antirequisites for {course_id.upper()}")
            for antireq_group in course_info["Antirequisites"]:
                for antireq in antireq_group:
                    rules.append(
                        f":- course({course_id.lower()}, {credits}, \"{career}\", {offered_spring1}, {offered_fall1}, {offered_spring2}, {offered_fall2}), \n   course({antireq.lower()}, _, \"{career}\", _, _, _, _). \n"
                    )

        # Generate prerequisite rules
        if course_info["Prerequisites"] != "NONE":
            rules.append(f"% Prerequisites for {course_id.upper()}")
            for prereq_group in course_info["Prerequisites"]:
                group_conditions = ", ".join(
                    [
                        f'\n   not course({prereq.lower()}, _, "{career}", _, _, _, _)'
                        for prereq in prereq_group
                    ]
                )
                rules.append(
                    f":- course({course_id.lower()}, {credits}, \"{career}\", {offered_spring1}, {offered_fall1}, {offered_spring2}, {offered_fall2}), {group_conditions}. \n" 
                )

        # Generate corequisite rules
        if course_info["Corequisites"] != "NONE":
            rules.append(f"% Corequisites for {course_id.upper()}")
            for coreq_group in course_info["Corequisites"]:
                for coreq in coreq_group:
                    rules.append(
                        f":- course({course_id.lower()}, {credits}, \"{career}\", {offered_spring1}, {offered_fall1}, {offered_spring2}, {offered_fall2}), \n   not course({coreq.lower()}, _, \"{career}\", {offered_spring1}, {offered_fall1}, {offered_spring2}, {offered_fall2}). \n"
                    )

    # Write predicates and rules to file
    output = (
        ["% Course predicates"] + predicates + ["\n% Course Rules \n"] + rules
    )
    _write_list_to_file(output, output_file)

    # Output results
    return {"predicates": predicates, "rules": rules}


# Usage
results = generate_clingo_predicates_rules(json_file)
# print("Predicates:")
# for predicate in results["predicates"]:
#     print(predicate)
# print("\nRules:")
# for rule in results["rules"]:
#     print(rule)

In [11]:
results

{'predicates': ['course(cse101, 3.0, "Undergraduate", 1, 1, 1, 1).',
  'course(cse102, 3.0, "Undergraduate", 0, 0, 0, 0).',
  'course(cse110, 3.0, "Undergraduate", 1, 1, 1, 1).',
  'course(cse113, 4.0, "Undergraduate", 0, 1, 0, 1).',
  'course(cse114, 4.0, "Undergraduate", 1, 0, 1, 0).',
  'course(cse130, 3.0, "Undergraduate", 1, 0, 1, 0).',
  'course(cse150, 4.0, "Undergraduate", 1, 1, 1, 1).',
  'course(cse160, 3.0, "Undergraduate", 1, 1, 1, 1).',
  'course(cse161, 1.0, "Undergraduate", 1, 1, 1, 1).',
  'course(cse190, 3.0, "Undergraduate", 1, 1, 1, 1).',
  'course(cse191, 3.0, "Undergraduate", 0, 1, 0, 1).',
  'course(cse192, 3.0, "Undergraduate", 0, 1, 0, 1).',
  'course(cse213, 4.0, "Undergraduate", 1, 1, 1, 1).',
  'course(cse214, 4.0, "Undergraduate", 0, 0, 0, 0).',
  'course(cse215, 4.0, "Undergraduate", 1, 1, 1, 1).',
  'course(cse216, 4.0, "Undergraduate", 1, 1, 0, 1).',
  'course(cse220, 4.0, "Undergraduate", 0, 1, 0, 0).',
  'course(cse230, 3.0, "Undergraduate", 1, 1, 0, 1)

In [12]:
tt = ['some text here \n']

In [26]:
tt[0].replace(' \n', '')

'some text here'